<a href="https://colab.research.google.com/github/Alexeyzhu/TriLossTensorboard/blob/master/TriLossTensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [0]:
# %reload_ext tensorboard

In [0]:
# import os
# logs_base_dir = "/home/vladbakhteev/projects/alex/runs"
# os.makedirs(logs_base_dir, exist_ok=True)
# %tensorboard --logdir {logs_base_dir}

In [0]:
# from tensorboard import notebook
# notebook.list()

Known TensorBoard instances:
  - port 6006: logdir runs (started 2 days, 19:38:32 ago; pid 15927)
  - port 6006: logdir runs (started 2 days, 20:39:26 ago; pid 5480)
  - port 6008: logdir /home/mmavlyutov/alex/runs (started 1 day, 5:25:19 ago; pid 3649)
  - port 6006: logdir runs (started 2 days, 20:02:36 ago; pid 10667)


In [0]:
import argparse
import os
import time
import io

import PIL.Image
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch import optim
from torch.autograd import Variable
from torch.backends import cudnn
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor

In [0]:
def get_loader(config):
    svhn_transform = transforms.Compose([
        transforms.Resize(config['image_size']),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    mnist_transform = transforms.Compose([
        transforms.Resize(config['image_size']),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    svhn = datasets.SVHN(root=config['svhn_path'], download=True, transform=svhn_transform)
    svhn_val = datasets.SVHN(root=config['svhn_path'], download=True, split='test', transform=svhn_transform)

    mnist = datasets.MNIST(root=config['mnist_path'], download=True, transform=mnist_transform)
    mnist_val = datasets.MNIST(root=config['mnist_path'], train=False, download=True, transform=mnist_transform)

    svhn_loader = torch.utils.data.DataLoader(dataset=svhn,
                                              batch_size=config['batch_size'],
                                              shuffle=True,
                                              num_workers=config['num_workers'])

    svhn_val_loader = torch.utils.data.DataLoader(dataset=svhn_val,
                                                  batch_size=500,
                                                  shuffle=False,
                                                  num_workers=config['num_workers'])

    mnist_loader = torch.utils.data.DataLoader(dataset=mnist,
                                               batch_size=config['batch_size'],
                                               shuffle=True,
                                               num_workers=config['num_workers'])

    mnist_val_loader = torch.utils.data.DataLoader(dataset=mnist_val,
                                                   batch_size=500,
                                                   shuffle=False,
                                                   num_workers=config['num_workers'])

    return svhn_loader, svhn_val_loader, mnist_loader, mnist_val_loader

In [0]:
class Net(nn.Module):
    def __init__(self, latent_dim=512):
        super(Net, self).__init__()
        
        conv_dim = 64
        self.conv1 = conv(3, conv_dim, 5, 2, 2)
        self.conv2 = conv(conv_dim, conv_dim * 2, 5, 2, 2)
        self.conv3 = conv(conv_dim * 2, conv_dim * 4, 5, 2, 2)
        self.conv4 = conv(conv_dim * 4, conv_dim * 8, 4, 1, 0)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 50)
        self.fc4 = nn.Linear(50, 10)

    def forward(self, x):
        # Encoder
        x = F.leaky_relu(self.conv1(x), 0.05)
        x = F.leaky_relu(self.conv2(x), 0.05)

        x = F.leaky_relu(self.conv3(x), 0.05)
        x = F.leaky_relu(self.conv4(x), 0.05)
        x = x.view(x.shape[0], -1)

        # Classifier
        x_c = F.relu(self.fc1(x))
        x_c = F.dropout(x_c, training=self.training)
        x_c = self.fc2(x_c)
        x_c = F.dropout(x_c, training=self.training)
        x_c = self.fc3(x_c)
        x_c = F.dropout(x_c, training=self.training)
        x_c = self.fc4(x_c)

        return x, F.softmax(x_c, dim=1)


class Net_D(nn.Module):
    def __init__(self, latent_dim=512):
        super(Net_D, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 50)
        self.fc4 = nn.Linear(50, 1)

    def forward(self, x):
        # Discriminator
        x_d = F.relu(self.fc1(x))
        x_d = F.dropout(x_d, training=self.training)
        x_d = self.fc2(x_d)
        x_d = F.dropout(x_d, training=self.training)
        x_d = self.fc3(x_d)
        x_d = F.dropout(x_d, training=self.training)
        x_d = self.fc4(x_d)

        return torch.sigmoid(x_d)


def conv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad, bias=True))  # bias=False
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)

In [0]:
def accuracy(true_labels, predicted_labels):
    _, pred = torch.max(predicted_labels, 1)

    correct = np.squeeze(pred.eq(true_labels.data.view_as(pred)))
    return correct.float().mean()


def xavier_weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        nn.init.constant_(m.bias, 0.1)


def pseudo_labeling(target, c, m=1):
    indexes = c[range(len(c)), torch.argmax(c, dim=1)] >= m
    indexes = np.array(np.nonzero(indexes.cpu().numpy())).flatten()
    pseudo_labels = c[indexes].clone()
    pseudo_labels = (pseudo_labels == pseudo_labels.max(dim=1, keepdim=True)[0]).long()
    _, pseudo_labels = torch.max(pseudo_labels, 1)
    uni = np.array(np.unique(pseudo_labels.cpu().numpy(), return_counts=True))
    mi = np.min(uni[1])
    if len(uni[1]) < 10:
        mi = 0
    ma = np.max(uni[1])
    return target[indexes], pseudo_labels, indexes, (mi + 1) / (ma + 1)

In [0]:
class Solver(object):
    def __init__(self, config, source_loader, source_val_loader, target_loader, target_val_loader):
        self.source_loader = source_loader
        self.source_val_loader = source_val_loader
        self.target_loader = target_loader
        self.target_val_loader = target_val_loader
        self.net = None
        self.net_optimizer = None
        self.net_d = None
        self.net_optimizer_d = None
        self.beta1 = config['beta1']
        self.beta2 = config['beta2']
        self.train_iters = config['train_iters']
        self.pretrain_iters = config['pretrain_iters']
        self.batch_size = config['batch_size']
        self.lr = config['lr']
        self.lr_d = config['lr_d']
        self.alpha_s = config['alpha_s']
        self.alpha_t = config['alpha_t']
        self.beta_c = config['beta_c']
        self.beta_sep = config['beta_sep']
        self.beta_p = config['beta_p']
        self.log_step = config['log_step']
        self.model_path = config['model_path']
        self.num_classes = config['num_classes']
        self.log_file = config['log_file']
        self.plot_path = config['plot_path']
        self.writer = SummaryWriter()
        self.build_model()

    def log(self, text):
        print(text)
        if self.log_file is not None:
            with open(self.log_file, 'a') as f:
                f.write(text + '\n')
                f.flush()
                f.close()

    def build_model(self):
        """Builds a generator and a discriminator."""
        self.net = Net()
        self.net_d = Net_D()

        net_params = list(self.net.parameters())
        net_d_params = list(self.net_d.parameters())
        self.net_optimizer = optim.Adamax(net_params, self.lr)
        self.net_optimizer_d = optim.Adamax(net_d_params, self.lr_d)

        if torch.cuda.is_available():
            self.net.cuda()
            self.net_d.cuda()

    def to_var(self, x):
        """Converts numpy to variable."""
        if torch.cuda.is_available():
            x = x.cuda()
        return Variable(x, requires_grad=False)

    def to_data(self, x):
        """Converts variable to numpy."""
        if torch.cuda.is_available():
            x = x.cpu()
        return x.data.numpy()

    def reset_grad(self):
        """Zeros the gradient buffers."""
        self.net_optimizer.zero_grad()
        self.net_optimizer_d.zero_grad()

    def separability_loss(self, labels, latents, imbalance_parameter=1):
        criteria = torch.nn.modules.loss.CosineEmbeddingLoss()
        loss_up = 0
        one_cuda = torch.ones(1).cuda()
        mean = torch.mean(latents, dim=0).cuda().view(1, -1)
        loss_down = 0
        for i in range(self.num_classes):
            indexes = labels.eq(i)
            mean_i = torch.mean(latents[indexes], dim=0).view(1, -1)
            if str(mean_i.norm().item()) != 'nan':
                for latent in latents[indexes]:
                    loss_up += criteria(latent.view(1, -1), mean_i, one_cuda)
                loss_down += criteria(mean, mean_i, one_cuda)
        loss = (loss_up / loss_down) * imbalance_parameter
        return loss

    def initialisation(self):
        self.net.apply(xavier_weights_init)
        self.net_d.apply(xavier_weights_init)
        source_iter = iter(self.source_loader)
        target_iter = iter(self.target_loader)
        target_val_iter = iter(self.target_val_loader)
        source_per_epoch = len(source_iter)
        target_per_epoch = len(target_iter)
        targetval_per_epoch = len(target_val_iter)
        self.log(f'{source_per_epoch}, {target_per_epoch}, {targetval_per_epoch}')

        criterion = nn.CrossEntropyLoss()

        f_labels = torch.LongTensor(128)
        f_labels[...] = 10

        t_labels = torch.LongTensor(128)
        t_labels[...] = 1

        # pretrain
        log_pre = 50
        source_iter = iter(self.source_loader)
        target_iter = iter(self.target_loader)
        return criterion, source_per_epoch, target_per_epoch, target_iter, source_iter, log_pre
    
        
    def train(self):
        
        # ============ Print latent ============#
        # print_latent(self.net, self.target_val_loader, self.source_val_loader,'before training', self.plot_path)
        
        criterion, source_per_epoch, target_per_epoch, target_iter, source_iter, log_pre = self.initialisation()

        pre_train = not os.path.exists(os.path.join(self.model_path, 'pre_train.pth'))

        # ============ Pretrain ============#
        if pre_train:
            self.log("Pretrain:\n*********")
            time_for_iter = []
            for step in range(self.pretrain_iters + 1):
                start_time = time.time()
                # ============ Initialization ============#
                # refresh
                if (step + 1) % (source_per_epoch) == 0:
                    source_iter = iter(self.source_loader)
                if (step + 1) % (target_per_epoch) == 0:
                    target_iter = iter(self.target_loader)
                # load the data
                source, s_labels = source_iter.next()
                
                target, _ = target_iter.next()
                
                target_rgb = target
                target = self.to_var(target_rgb)
                source, s_labels = self.to_var(source), self.to_var(s_labels).long().squeeze()

                # ============ Training ============ #
                self.reset_grad()
                # forward
                latent, c = self.net(source)
                # loss
                loss_source_class = criterion(c, s_labels)

                # one step
                loss_source_class.backward()
                self.net_optimizer.step()
                self.reset_grad()
                # ============ Validation ============ #
                if (step + 1) % log_pre == 0:
                    _, c_source = self.net(source)
                    _, c_target = self.net(target)
                    self.log("[%d/%d] classification loss: %.4f; source accuracy  %.4f" % (
                        step + 1,self.pretrain_iters,  loss_source_class.item(), accuracy(s_labels, c_source)))
                    
                    self.writer.add_scalar('Classification loss/Pretrain Loss', loss_source_class.item(), step)
                    self.writer.add_scalar('Pretrain Accuracy/source-train', accuracy(s_labels, c_source), step)
                    tar_acc, sour_acc = self.validate()
                    self.writer.add_scalar('Pretrain Accuracy/target-test', tar_acc, step)
                    self.writer.add_scalar('Pretrain Accuracy/source-test', sour_acc, step)
                    
                    self.log('Pretrain time remaining %.1f min' %(
                        (sum(time_for_iter) / len(time_for_iter)) * (self.pretrain_iters - step) / 60))
                    self.writer.flush()
                    
                time_for_iter.append(time.time() - start_time)
            # ============ Print latent ============#
            self.save_model()
        else:
            self.load_model()
        
        #print_latent(self.net, self.target_val_loader, self.source_val_loader, 'after pretraining', self.plot_path)

        # ============ Initialization ============ #
        source_iter = iter(self.source_loader)
        target_iter = iter(self.target_loader)
        maxacc = 0.0
        maximum_acc = 0.0
        max_iter = 0
        net_params = list(self.net.parameters())
        net_d_params = list(self.net_d.parameters())

        self.net_optimizer = optim.Adam(net_params, self.lr, [self.beta1, self.beta2])
        self.net_optimizer_d = optim.Adam(net_d_params, self.lr_d, [self.beta1, self.beta2])
        
        self.log("Second:\n******")
        
        time_for_iter = []
        for step in range(self.train_iters):
            start_time = time.time()
            # ============ Initialization ============#

            # refresh
            if (step + 1) % (target_per_epoch) == 0:
                target_iter = iter(self.target_loader)
            if (step + 1) % (source_per_epoch) == 0:
                source_iter = iter(self.source_loader)
            # load the data
            source, s_labels = source_iter.next()
            source, s_labels = self.to_var(source), self.to_var(s_labels).long().squeeze()  # must squeeze
            target, _ = target_iter.next()
            target_rgb = target
            target = self.to_var(target_rgb)

            # ============ train D ============#
            self.reset_grad()

            latent_source, c = self.net(source)
            
            d = self.net_d(latent_source)
            loss_d_s1 = F.binary_cross_entropy(d, torch.ones_like(d, dtype=torch.float32))
            loss_d_s0 = F.binary_cross_entropy(d, torch.zeros_like(d, dtype=torch.float32))
            loss_c_source = criterion(c, s_labels)

            latent_target, c = self.net(target)
            
            d = self.net_d(latent_target)
            loss_d_t0 = F.binary_cross_entropy(d, torch.zeros_like(d, dtype=torch.float32))

            loss_p = loss_d_s0
            loss_d = loss_d_s1 + loss_d_t0
            # ============ train pseudo labeling ============#

            chosen_target, pseudo_labels, indexes, imbalance_parameter = pseudo_labeling(target, c)

            if chosen_target is not None:
                loss_c_target = criterion(c[indexes], pseudo_labels)

                latent_target = latent_target[indexes]
                # ============ class loss  ============#
                loss_sep = self.separability_loss(torch.cat((s_labels, pseudo_labels)),
                                                  torch.cat((latent_source, latent_target)),
                                                  imbalance_parameter=imbalance_parameter)
            else:
                loss_c_target = 0
                loss_sep = 0
            loss = self.beta_c * (self.alpha_s * loss_c_source + self.alpha_t * loss_c_target) + \
                   self.beta_p * loss_p + \
                   self.beta_sep * loss_sep

            loss.backward(retain_graph=True)
            self.net_optimizer.step()

            loss_d.backward()
            self.net_optimizer_d.step()

            self.reset_grad()

            # ============ Validation ============ #
            if (step + 1) % self.log_step == 0:
                _, c_source = self.net(source)
                _, c_target = self.net(target)
                
                self.log(f'Iteration [{step+1} / {self.train_iters}]')
                self.log("Max accuracy: %.2f on iteration %d; Current source  accuracy  %.4f" 
                         % (maximum_acc, max_iter, accuracy(s_labels, c_source)))
                
                self.writer.add_scalar('Classification loss/Train Loss', loss.item(), step)
                self.writer.add_scalar('Train Accuracy/source-train', accuracy(s_labels, c_source), step)
                acc, sour_acc = self.validate()
                self.writer.add_scalar('Train Accuracy/target-test', acc, step)
                self.writer.add_scalar('Train Accuracy/source-test', sour_acc, step)
                self.writer.flush()
                
                if acc > maximum_acc:
                    maximum_acc = acc
                    max_iter = step
                    
                self.log('Train time remaining %.1f min' %(
                    sum(time_for_iter) / len(time_for_iter) * (self.train_iters - step) / 60))
                
            time_for_iter.append(time.time() - start_time)
            
            self.reset_grad()
            
        self.validate(True)
        #============Print latent space============#
        # print_latent(self.net, self.target_val_loader, self.source_val_loader, 'after training', self.plot_path)
        
        # ============ Save the model ============ #
        torch.save(self.net, "./model_c_final.pth")
        torch.save(self.net_d, "./model_d_final.pth")
        self.writer.close()

    def validate(self, details = False):
        class_correct = [0] * self.num_classes
        class_total = [0.] * self.num_classes
        classes = [str(i) for i in range(self.num_classes)]
        self.net.eval()  # prep model for evaluation

        for data, target in self.target_val_loader:

            # forward pass: compute predicted outputs by passing inputs to the model
            data, target = self.to_var(data), self.to_var(target).long().squeeze()

            data = data.cuda()
            target = target.cuda()

            latent, output = self.net(data)
            _, pred = torch.max(output, 1)
            correct = np.squeeze(pred.eq(target.data.view_as(pred)))
            # calculate test accuracy for each object class
            for i in range(len(target.data)):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
        
        if details:
            for i in range(self.num_classes):
                if class_total[i] > 0:
                    self.log('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                        str(i), 100 * class_correct[i] / class_total[i],
                        np.sum(class_correct[i]), np.sum(class_total[i])))
                else:
                    self.log('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
            
        target_accuracy = 100. * np.sum(class_correct) / np.sum(class_total)
        
        self.log("\nTest Target Accuracy (Overall) [%d / %d]: %2d%%" 
                 % (np.sum(class_correct), np.sum(class_total), target_accuracy))
        
        class_correct = [0] * self.num_classes
        class_total = [0.] * self.num_classes
        classes = [str(i) for i in range(self.num_classes)]
        
        for data, target in self.source_val_loader:

            # forward pass: compute predicted outputs by passing inputs to the model
            data, target = self.to_var(data), self.to_var(target).long().squeeze()

            data = data.cuda()
            target = target.cuda()

            latent, output = self.net(data)
            _, pred = torch.max(output, 1)
            correct = np.squeeze(pred.eq(target.data.view_as(pred)))
            # calculate test accuracy for each object class
            for i in range(len(target.data)):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
        
        source_accuracy = 100. * np.sum(class_correct) / np.sum(class_total)
        
        self.log("\nTest Source Accuracy (Overall) [%d / %d]: %2d%%" 
                 % (np.sum(class_correct), np.sum(class_total), source_accuracy))
        
        self.net.train()
        return target_accuracy, source_accuracy

    def save_model(self):
        torch.save(self.net, os.path.join(self.model_path, 'pre_train.pth'))

    def load_model(self):
        self.net = torch.load(os.path.join(self.model_path, 'pre_train.pth'))

In [0]:
def print_latent(model, target_val_loader, source_val_loader, title, plot_path):
        
        target, t_labels = get_all_data(target_val_loader)
        
        latent, _ = model(target.cuda())
        plot_latent_space(plot_path, latent.detach().cpu().numpy(), t_labels.numpy(), 'Target ' + title)
        
        source, s_labels = get_all_data(source_val_loader, MNIST = False)
            
        latent, _ = model(source.cuda())
        plot_latent_space(plot_path, latent.detach().cpu().numpy(), s_labels.numpy(), 'Source ' + title)
        
        
        latent, _ = model(torch.cat((target, source),0).cuda())
        
        plot_latent_space(
            plot_path, latent.detach().cpu().numpy(), 
            torch.cat((t_labels, s_labels), 0).cpu().numpy(), 
            'Target and Source ' + title, 
            torch.cat((
                torch.zeros((t_labels.size()[0], )), 
                torch.ones((s_labels.size()[0], ))), 0))

In [0]:
def get_all_data(data_loader, MNIST = True):
    labels = None
    images = None
    i = 0
    
    for data, target  in data_loader:
        if i > 9:
            break
        data, target = data.cpu(), target.cpu()
        if labels is None:
            labels = target
            images = data
        else:
            labels = torch.cat((labels, target), 0)
            images = torch.cat((images, data), 0)
            
        i+=1

    return images, labels

In [0]:
def plot_latent_space(plot_path, X, y, title, dataset = None):
    
    print(f'Images shape: {X.shape}')
    print(f'Labels shape: {y.shape}')

    feat_cols = [ 'pixel' + str(i) for i in range(X.shape[1]) ]
    df = pd.DataFrame(X,columns=feat_cols)
    df['y'] = y
    
    if dataset is not None:
        dataset = dataset.cpu().numpy()
        df['class'] = dataset
        
    df['label'] = df['y'].apply(lambda i: str(i))
    X, y = None, None
    
    print('Size of the dataframe: {}'.format(df.shape))

    rndperm = np.random.permutation(df.shape[0])
    
    N = 10000

    df_subset = df.loc[rndperm[:N],:].copy()
    data_subset = df_subset[feat_cols].values

    time_start = time.time()
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(data_subset)
    print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

    df_subset['tsne-2d-one'] = tsne_results[:,0]
    df_subset['tsne-2d-two'] = tsne_results[:,1]

    plt.figure(figsize=(10,10))
    
    plt.title(title)

    
    sns_plot = sns.scatterplot(
        x="tsne-2d-one", y="tsne-2d-two",
        hue="y",
        palette=sns.hls_palette(10, l=.5, s=.5),
        data=df_subset,
        legend="full",
        alpha=0.99
    )
    plt.savefig(f'{plot_path}/{title}.png')
    plt.show()
    
    if dataset is not None:
        plt.figure(figsize=(10,10))

        plt.title(title)


        sns_plot = sns.scatterplot(
            x="tsne-2d-one", y="tsne-2d-two",
            hue="class",
            palette=sns.hls_palette(2, l=.5, s=.5),
            data=df_subset,
            legend="full",
            alpha=0.99
        )
        plt.savefig(f'{plot_path}/{title}_dataset.png')
        plt.show()

In [0]:
def main(conf):
    os.makedirs(conf['plot_path'], exist_ok=True)
    
    svhn_loader, svhn_val_loader, mnist_loader, mnist_val_loader = get_loader(conf)

    solver = Solver(conf, svhn_loader, svhn_val_loader, mnist_loader, mnist_val_loader)
    cudnn.benchmark = True

    # create directories if not exist
    if not os.path.exists(conf['model_path']):
        os.makedirs(conf['model_path'])
    if conf['mode'] == 'train':
        solver.train()

def set_seed(seed):
    """
    Enables reproducibility of results of torch project
    :param seed: seed for randomizers
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

def start():
    # for reproducibility
    set_seed(42)
    
    parser = {}
    # model hyper-parameters
    parser.update({'image_size': 32})
    parser.update({'num_classes': 10})
    parser.update({'alpha_s': 0.5})
    parser.update({'alpha_t': 0.8})
    parser.update({'beta_c': 1})
    parser.update({'beta_sep': 1.5})
    parser.update({'beta_p': 4})

    # training hyper-parameters
    parser.update({'train_iters': 1000})
    parser.update({'pretrain_iters': 5000})
    parser.update({'batch_size': 1024})
    parser.update({'num_workers': 2})
    parser.update({'lr': 0.001})
    parser.update({'lr_d': 0.001})
    parser.update({'beta1': 0.5})
    parser.update({'beta2': 0.999})

    # misc
    parser.update({'mode': 'train'})
    parser.update({'model_path': './models'})
    parser.update({'plot_path': './plots'})
    parser.update({'mnist_path': './data/mnist'})
    parser.update({'svhn_path': './data/svhn'})
    parser.update({'log_file': './out.txt'})
    parser.update({'log_step': 20})

    # clean up log file
    with open(parser['log_file'], 'w') as f:
        f.close()
    
    main(parser)

start()

Using downloaded and verified file: ./data/svhn/train_32x32.mat
Using downloaded and verified file: ./data/svhn/test_32x32.mat
72, 59, 20
Pretrain:
*********
[50/5000] classification loss: 2.0229; source accuracy  0.4268

Test Target Accuracy (Overall) [2512 / 10000]: 25%

Test Source Accuracy (Overall) [10498 / 26032]: 40%
Pretrain time remaining 13.7 min
[100/5000] classification loss: 1.8734; source accuracy  0.5820

Test Target Accuracy (Overall) [3687 / 10000]: 36%

Test Source Accuracy (Overall) [14709 / 26032]: 56%
Pretrain time remaining 15.9 min
[150/5000] classification loss: 1.8047; source accuracy  0.6738

Test Target Accuracy (Overall) [4060 / 10000]: 40%

Test Source Accuracy (Overall) [17477 / 26032]: 67%
Pretrain time remaining 16.9 min
[200/5000] classification loss: 1.6944; source accuracy  0.7598

Test Target Accuracy (Overall) [4832 / 10000]: 48%

Test Source Accuracy (Overall) [19306 / 26032]: 74%
Pretrain time remaining 17.0 min
[250/5000] classification loss: 1.6

/home/mmavlyutov/alex/venv/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/mmavlyutov/alex/venv/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/mmavlyutov/alex/venv/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/mmavlyutov/alex/venv/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + ob

Second:
******
Iteration [20 / 1000]
Max accuracy: 0.00 on iteration 0; Current source  accuracy  0.9512

Test Target Accuracy (Overall) [8897 / 10000]: 88%

Test Source Accuracy (Overall) [21232 / 26032]: 81%
Train time remaining 32.4 min
Iteration [40 / 1000]
Max accuracy: 88.97 on iteration 19; Current source  accuracy  0.9551

Test Target Accuracy (Overall) [8897 / 10000]: 88%

Test Source Accuracy (Overall) [22266 / 26032]: 85%
Train time remaining 34.0 min
Iteration [60 / 1000]
Max accuracy: 88.97 on iteration 19; Current source  accuracy  0.9619

Test Target Accuracy (Overall) [9276 / 10000]: 92%

Test Source Accuracy (Overall) [22394 / 26032]: 86%
Train time remaining 34.2 min
Iteration [80 / 1000]
Max accuracy: 92.76 on iteration 59; Current source  accuracy  0.9648

Test Target Accuracy (Overall) [9065 / 10000]: 90%

Test Source Accuracy (Overall) [22561 / 26032]: 86%
Train time remaining 33.9 min
Iteration [100 / 1000]
Max accuracy: 92.76 on iteration 59; Current source  acc

/home/mmavlyutov/alex/venv/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net_D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
